## Análisis de las flores Iris

Las flores Iris son una familia de flores como se describe en https://es.wikipedia.org/wiki/Iris_flor_conjunto_de_datos.

Podemos encontrar la base de datos en formato csv en http://badillosoft.com/iris.csv. La cuál contiene variarias columnas como `sepal length`, `sepal width`, `petal length`, `petal width` y `class`.

Las primeras 4 columnas indican características de la flor Iris y la última columna representa la clase (especie) a la que pertenece la flor Iris. Es decir, las primeras 4 columnas serán utilizadas como las entradas para la clasificación y la última columna será utilizada como la salida de la clasificación.

Por lo tanto, queremos construir una red neuronal que aprenda a clasificar a qué especie pertenece una flor Iris, dados sus tamaños de sépalo y pétalo y que nos diga a que especie o variedad pertenece.

Para conseguirlo debemos entrenar a la red neuronal con los datos reales de la base de datos, siguiendo la norma que 80% de los ejemplos sean para el entrenamiento y 20% de los ejemplos se utilicen para medir el desempeño del aprendizaje. Recordando que en este caso particular los datos vienen ordenados, por lo que antes, tendremos que reordenarlos aleatoriamente para que no haya pérdida de generalidad (si tomamos los primeros 80% de los datos ordenados el 20% restante serían sólo de la clase virgínica por lo que no sería un buen entrenamiento).

Además el pre-procesamiento debe incluir una columna extra dónde las clases sean categóricas y no cadenas de texto como vienen los datos.

In [21]:
import pandas as pd

df = pd.read_csv("http://badillosoft.com/iris.csv")

print(df.head(3))
print(df.tail(3))

   sepal length  sepal width  petal length  petal width        class
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
     sepal length  sepal width  petal length  petal width           class
147           6.5          3.0           5.2          2.0  Iris-virginica
148           6.2          3.4           5.4          2.3  Iris-virginica
149           5.9          3.0           5.1          1.8  Iris-virginica


In [22]:
df["especie"] = df["class"].map({
    "Iris-setosa": 1,
    "Iris-versicolor": 2,
    "Iris-virginica": 3
})

print(df.head(3))
print(df.tail(3))

   sepal length  sepal width  petal length  petal width        class  especie
0           5.1          3.5           1.4          0.2  Iris-setosa        1
1           4.9          3.0           1.4          0.2  Iris-setosa        1
2           4.7          3.2           1.3          0.2  Iris-setosa        1
     sepal length  sepal width  petal length  petal width           class  \
147           6.5          3.0           5.2          2.0  Iris-virginica   
148           6.2          3.4           5.4          2.3  Iris-virginica   
149           5.9          3.0           5.1          1.8  Iris-virginica   

     especie  
147        3  
148        3  
149        3  


In [23]:
df = df.sample(frac=1)

print(df.head())

     sepal length  sepal width  petal length  petal width           class  \
8             4.4          2.9           1.4          0.2     Iris-setosa   
103           6.3          2.9           5.6          1.8  Iris-virginica   
122           7.7          2.8           6.7          2.0  Iris-virginica   
121           5.6          2.8           4.9          2.0  Iris-virginica   
13            4.3          3.0           1.1          0.1     Iris-setosa   

     especie  
8          1  
103        3  
122        3  
121        3  
13         1  


Hasta ahora tenemos un dataframe con 6 columnas `sepal length`, `sepal width`, `petal length`, `petal width`, `class`, `especie`.

Para entrenar una red neuronal necesitamos definir las columnas que serán la entrada de la red, en este caso serán las columnas `sepal length`, `sepal width`, `petal length`, `petal width`. A esta matriz de entrenamiento se le llama el  `x_train`. Recordando que para entrenar la red neuronal necesitamos sólo el 80% de los datos, es decir, si temos `150` datos, necesitamos `0.8 * 150` datos de entrenamiento. El 20% de datos restante será utilizado para medir el desempeño de la red neuronal, a este último conjunto de muestras se le llama `x_test`.

Primero recuperamos las columnas que serán utilizadas para el entrenamiento y el desempeño, a estos datos se les conoce como la muestra de entrada `x`.

In [37]:
x = df.filter(items=["sepal length", "sepal width", "petal length", "petal width"])

print(x.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 8 to 124
Data columns (total 4 columns):
sepal length    150 non-null float64
sepal width     150 non-null float64
petal length    150 non-null float64
petal width     150 non-null float64
dtypes: float64(4)
memory usage: 5.9 KB
None


Luego definimos el número de muestras para entrenar, es decir `k = 0.8 * n` donde `n` es el número de total muestras y `k` el 80% de ellas.

In [30]:
n = len(x)
k = int(n * 0.8)

print("Se tomarán {}/{} muestras para el entrenamiento".format(k, n))

Se tomarán 120/150 muestras para el entrenamiento


Obtenemos las primeras `k` muestas del total de muestras y eso es `x_train`.

In [33]:
x_train = x[:k] # Las primeras 120 muestras desde 0 a 119

print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 8 to 17
Data columns (total 4 columns):
sepal length    120 non-null float64
sepal width     120 non-null float64
petal length    120 non-null float64
petal width     120 non-null float64
dtypes: float64(4)
memory usage: 4.7 KB
None


Obtenemos las restantes muestras para medir el desempeño y eso es `x_test`.

In [34]:
x_test = x[k:] # Las últimas 30 muestras desde 120 a 149

print(x_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 56 to 124
Data columns (total 4 columns):
sepal length    30 non-null float64
sepal width     30 non-null float64
petal length    30 non-null float64
petal width     30 non-null float64
dtypes: float64(4)
memory usage: 1.2 KB
None


Una vez que hemos definido las muestras para el entrenamiento (`x_train`) y las muestras para medir el desempeño (`x_test`), ahora debemos obtener los objetivos para el entrenamiento (`y_train`) y los objetivos para medir el desempeño (`y_test`) de forma similar.

In [40]:
y = df.filter(items=["especie"])

y_train = y[:k]

y_test = y[k:]

print(y.info())
print(y_train.info())
print(y_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 8 to 124
Data columns (total 1 columns):
especie    150 non-null int64
dtypes: int64(1)
memory usage: 2.3 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 8 to 17
Data columns (total 1 columns):
especie    120 non-null int64
dtypes: int64(1)
memory usage: 1.9 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 56 to 124
Data columns (total 1 columns):
especie    30 non-null int64
dtypes: int64(1)
memory usage: 480.0 bytes
None


Hasta ahora hemos formado las matrices `x_train`, `x_test`, `y_train` y `y_test`. De cuales `x_train` y `y_train` seán utilizadas por la red neuronal para realizar el entrenamiento y `x_test` y `y_test` serán utilizadas para evaluar el desempeño del entrenamiento.

Entonces, utilizando Keras crearemos una red neuronal de tipo ANN con la topología `4-8-2-1` con las funciones de activación `Tanh-Tanh-Tanh-ReLU`.

In [98]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(4, activation="tanh", input_dim=4))
model.add(Dense(8, activation="tanh"))
model.add(Dense(2, activation="tanh"))
model.add(Dense(1, activation="relu"))

model.compile(optimizer="adam", loss="mse", metrics=["accuracy", "categorical_accuracy"])

print(model)

Ahora que ya tenemos nuestro modelo de la red neuronal ANN podemos entrenar a la red para que ajuste sus pesos mediante `x_train` y `y_train`.

In [134]:
model.fit(x_train.values, y_train.values, epochs=10, batch_size=10)

Epoch 1/10
120/120 [==============================] - 0s 313us/step - loss: 0.0395 - acc: 0.9750 - categorical_accuracy: 1.0000
Epoch 2/10
120/120 [==============================] - 0s 363us/step - loss: 0.0390 - acc: 0.9750 - categorical_accuracy: 1.0000
Epoch 3/10
120/120 [==============================] - 0s 282us/step - loss: 0.0399 - acc: 0.9750 - categorical_accuracy: 1.0000
Epoch 4/10
120/120 [==============================] - 0s 271us/step - loss: 0.0390 - acc: 0.9750 - categorical_accuracy: 1.0000
Epoch 5/10
120/120 [==============================] - 0s 248us/step - loss: 0.0395 - acc: 0.9750 - categorical_accuracy: 1.0000
Epoch 6/10
120/120 [==============================] - 0s 387us/step - loss: 0.0399 - acc: 0.9750 - categorical_accuracy: 1.0000
Epoch 7/10
120/120 [==============================] - 0s 342us/step - loss: 0.0390 - acc: 0.9750 - categorical_accuracy: 1.0000
Epoch 8/10
120/120 [==============================] - 0s 300us/step - loss: 0.0412 - acc: 0.9750 - categ

Ahora que el modelo ya está ajustada (la red neuronal ya aprendió los ejemplos de entrenamiento) podemos medir el desempeño mediante `x_test` y `y_test`.

In [135]:
metrics = model.evaluate(x_test.values, y_test.values)

print(metrics)

30/30 [==============================] - 0s 72us/step
[0.0059516639448702335, 1.0, 1.0]


In [136]:
import numpy as np

for i in range(len(y_test)):
    xt = x_test.values[i]
    yt = y_test.values[i]
    yp = model.predict(np.array([xt]))
    print("{} => {} <> {}".format(xt, yt, yp))
    

[6.3 3.3 4.7 1.6] => [2] <> [[2.0526638]]
[5.1 3.3 1.7 0.5] => [1] <> [[1.104357]]
[7.2 3.6 6.1 2.5] => [3] <> [[2.9728053]]
[6.  2.9 4.5 1.5] => [2] <> [[2.1977637]]
[4.9 3.1 1.5 0.1] => [1] <> [[1.0708456]]
[5.  2.3 3.3 1. ] => [2] <> [[1.7901336]]
[5.8 2.8 5.1 2.4] => [3] <> [[3.0285578]]
[6.1 3.  4.6 1.4] => [2] <> [[2.0896974]]
[6.7 3.1 5.6 2.4] => [3] <> [[3.0079072]]
[5.7 4.4 1.5 0.4] => [1] <> [[0.98218316]]
[5.2 3.4 1.4 0.2] => [1] <> [[1.0311562]]
[5.1 3.5 1.4 0.2] => [1] <> [[1.0164591]]
[6.4 3.2 4.5 1.5] => [2] <> [[1.9525874]]
[4.6 3.2 1.4 0.2] => [1] <> [[1.0167408]]
[5.5 2.4 3.7 1. ] => [2] <> [[1.8654542]]
[6.7 3.3 5.7 2.5] => [3] <> [[3.0057142]]
[4.6 3.4 1.4 0.3] => [1] <> [[0.9958205]]
[7.7 2.6 6.9 2.3] => [3] <> [[3.045065]]
[6.8 3.  5.5 2.1] => [3] <> [[2.9348764]]
[6.9 3.2 5.7 2.3] => [3] <> [[2.9718678]]
[6.7 3.1 4.4 1.4] => [2] <> [[1.8900932]]
[7.1 3.  5.9 2.1] => [3] <> [[2.9789279]]
[6.5 3.  5.8 2.2] => [3] <> [[3.013321]]
[5.1 3.7 1.5 0.4] => [1] <> [[1.0172